In [1]:
import numpy as np
from hmmlearn import hmm
import alfabeto_data.dissertation_images as di
from alfabeto_sources.all_sources import *
from alfabeto_code.AlfabetoConverter import transposed_pc_chords_noMMD
from Continuo.ContinuoConverter import figure_intervals_pc
import csv
import copy
from scipy.spatial.distance import euclidean, cdist
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
from sklearn.externals import joblib

ready!


In [2]:
# X = di.python_hmm_corpus(di.monteverdi_continuo, di.modal_major)
X = di.python_hmm(di.GetAll.all_alf, di.modal_major, 'chords')

In [3]:
X[2]

['I',
 'i',
 'II',
 'ii',
 'bIII',
 'III',
 'iii',
 'IV',
 'iv',
 'V',
 'v',
 'bVI',
 'VI',
 'vi',
 'bVII',
 'bvii',
 'VII',
 'vii']

In [85]:
#compares emission probability
from alfabeto_sources.all_sources import composer_labels_linebreak

def emission_comparison():
    composer_data = {}
    composer_scores = {}
    for name, book in composer_labels_linebreak.items():
#         if 'Kapsperger' in name:
        print('starting', name)
        X = di.python_hmm(book, di.modal_major, 'chords')
        all_numerals = ['I', 'i', 'bII', 'bii', 'II', 'ii', 'bIII', 'biii', 'III', 'iii', 'IV', 'iv',
                        '#IV', '#iv', 'V', 'v', 'bVI', 'bvi', 'VI', 'vi', 'bVII', 'bvii', 'VII', 'vii']
        CORPUS_NUMERALS = X[2]
        Z = hmm.MultinomialHMM(n_components=3, verbose=False,n_iter=1000,tol=.0001)
        Z.n_features = len(X[2])
        FIT_DATA = Z.fit(X[0], X[1])
        composer_scores[name] = FIT_DATA.score(X[0], X[1])
        EMIT = FIT_DATA.emissionprob_
        indices = []
        for a in all_numerals:
            if a not in CORPUS_NUMERALS:
                indices.append(all_numerals.index(a))

        for n in indices:
            EMIT = np.insert(EMIT, n, 0, axis=1)

        full_array = [] #flattened, ordered array of emission data
        I = EMIT[:,all_numerals.index("I")]
        IV = EMIT[:,all_numerals.index("IV")]
        V = EMIT[:,all_numerals.index("V")]
        for x in EMIT[np.argmax(I)]:
            full_array.append(x)
        for x in EMIT[np.argmax(IV)]:
            full_array.append(x)
        for x in EMIT[np.argmax(V)]:
            full_array.append(x)
        composer_data[name] = full_array
    joblib.dump(composer_data, '/home/daniel/Desktop/composer_data.pkl', compress=9)
    print(composer_scores)
    return composer_data

In [86]:
composer_data_compare = emission_comparison()
all_labels = []
all_data = []
all_data_measured = []
for a, b in composer_data_compare.items():
    all_labels.append(a)
    all_data.append(b)
DATA = cdist(all_data, all_data, 'euclidean')
# for x in all_data:
#     for y in all_data:
#         all_data_measured.append(euclidean(x, y))

starting Obizzi
1627


/home/daniel/anaconda3/lib/python3.5/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/home/daniel/anaconda3/lib/python3.5/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),


starting Milanuzzi
1635
starting Marini
1622


/home/daniel/anaconda3/lib/python3.5/site-packages/hmmlearn/hmm.py:405: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T


starting Vitali
1622
starting Kapsperger
1630
starting Kapsperger
1610
starting Stefani
1623
starting Milanuzzi
1628
starting Ghizzolo
1623
starting Rontani
1620
starting Kapsperger
1632
starting Kapsperger
1640
starting Kapsperger
1619a
starting DIndia
1623
starting Stefani
1622
starting Milanuzzi
1622
starting Kapsperger
1623
starting Milanuzzi
1630
starting Sabbatini
1652
starting Rontani
1619
starting Rontani
1623
starting Giaccio
1618
starting Landi
1620
starting Giaccio
1620
starting Kapsperger
1619b
starting DIndia
1621
starting Stefani
1621
starting Montesardo
1612
starting Milanuzzi
1625
{'DIndia\n1621': -102.54862623823456, 'Milanuzzi\n1635': -657.01794885253798, 'Marini\n1622': -724.95555621020117, 'Stefani\n1623': -1373.9435072018086, 'Kapsperger\n1630': -1305.5504774618607, 'Kapsperger\n1610': -779.97152495318903, 'Vitali\n1622': -433.1681814242138, 'Milanuzzi\n1628': -537.48905669212002, 'Ghizzolo\n1623': -1222.7137920317107, 'Rontani\n1620': -54.622330430023894, 'Kapsper

In [68]:
# di.neato_plain(DATA*100, all_labels, '/home/daniel/Desktop/neatofunk.pdf')
def neato_plain(list_of_lists, label_lists, path):
    dt = [('len', float)]

    A = (np.array(list_of_lists)/10).view(dt)
    # print(A)
    G = nx.from_numpy_matrix(A)
    relabeled_nodes = {}
    for x, y in zip(range(len(label_lists)), label_lists):
        relabeled_nodes[x] = y
        # print(y[1:-1])
    G = nx.relabel_nodes(G, relabeled_nodes)

    G = to_agraph(G)

    G.node_attr.update(shape='none', fontsize=12)
    G.edge_attr.update(color='black', style='invis')
    #  ♭ ♮ ...for copy and paste

    G.draw(path, format='pdf', prog='neato')
neato_plain(DATA*100, all_labels, '/home/daniel/Desktop/neatofunk.pdf')

In [ ]:
Z = hmm.MultinomialHMM(n_components=3, verbose=True,n_iter=1000,tol=.01)
Z.n_features = len(X[2])
FIT_DATA = Z.fit(X[0], X[1])

In [ ]:
EMIT = FIT_DATA.emissionprob_

In [ ]:
CORPUS_NUMERALS = X[2]

In [ ]:
all_numerals = ['I', 'i', 'bII', 'bii', 'II', 'ii', 'bIII', 'biii', 'III', 'iii', 'IV', 'iv',
                '#IV', '#iv', 'V', 'v', 'bVI', 'bvi', 'VI', 'vi', 'bVII', 'bvii', 'VII', 'vii']

indices = []
for a in all_numerals:
    if a not in CORPUS_NUMERALS:
        print(a)
        indices.append(all_numerals.index(a))

for n in indices:
    EMIT = np.insert(EMIT, n, 0, axis=1)


In [17]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
np.argmax(a[0])

2

In [ ]:
emission_prob = []
for x in ESTEMIT11:
    estlist = []
    for y in x:
        if y >= .01:
            hey.append(y)
        else:
            hey.append(0)
    emission_prob.append(estlist)

In [17]:
def csv_function(f_variable, path):
    with open(path, 'w', newline='') as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(f_variable)

In [ ]:
csv_function(emission_prob, '/home/daniel/Desktop/emit_moz_maj.csv')

In [ ]:
#compute marginal distribution
trn = copy.deepcopy(ESTTR11).conj().transpose()
(D,V) = copy.deepcopy(np.linalg.eig(trn))
item=max(D)
idx = 0
for x, y in zip(D, range(len(D))):
    if x == item:
        idx = y
f_marginal=abs(V[:,idx])/sum(abs(V[:,idx]))
#whew!

#compute P(F|C)
k = []
for x in ESTEMIT11:
    k.append(np.zeros(np.size(x)))
PFC = np.array(k)
for jj in range(1,len(ESTEMIT11[0])+1):
    for ii in range(1,len(ESTEMIT11[:,0])+1):
        PFC[ii-1][jj-1]=ESTEMIT11[ii-1][jj-1]*f_marginal[ii-1]/jj
    PFC[:,jj-1]=PFC[:,jj-1]/sum(PFC[:,jj-1])
    b = PFC[:,jj-1]
    a = sum(PFC[:,jj-1])

In [ ]:
#Most Likely P(F|C)
[midx,_] = PFC.max(0),PFC.argmax(0)
state_dict = {}
for kk in range(K):
    state_dict[kk] = {}
    for ll in range(len(midx)):
        if PFC[kk][ll]>0.01:
            state_dict[kk][TOKEN_NAMES[ll]]=PFC[kk][ll]


In [ ]:
CSV_LIST = [TOKEN_NAMES]
for state in range(len(state_dict)):
    token_pct = []
    for t in TOKEN_NAMES:
        num = 0
        for a,b in state_dict[state].items():
            if t == a:
                token_pct.append(b)
                num+=1
        if num == 0:
            token_pct.append(0)
    CSV_LIST.append(token_pct)

In [ ]:
csv_function(CSV_LIST, '/home/daniel/Desktop/alf_major_functions.csv')

In [ ]:
emission_prob = copy.deepcopy(ESTEMIT11)
new_csv = copy.deepcopy(np.array(CSV_LIST[1:]))
# print(TOKEN_NAMES)
# print(emission_prob.shape)
token_reduced = []
delete = []
for y in range(len(emission_prob[0])):
    if sum(emission_prob[0:,y]) <= .03:
        delete.append(y)
    else:
        token_reduced.append(TOKEN_NAMES[y])
emission_reduced = np.delete(new_csv, delete, axis=1)
# print(emission_reduced.shape)
# print(len(token_reduced))
# print(token_reduced)
# print(emission_reduced)

In [ ]:
reduced_csv = [token_reduced]
for x in emission_reduced:
    reduced_csv.append(x)
csv_function(reduced_csv, '/home/daniel/Desktop/reduced_pal_major4.csv')

In [ ]:
# emission = np.swapaxes(np.array(CSV_LIST[1:]), 0, 1)
# # emission = np.array(emission_reduced)
# neato_data = cdist(emission, emission, 'euclidean')*10
# # neato_data = []
# # for x in emission:
# #     temp = []
# #     for y in emission:
# #         temp.append(euclidean(x, y))
# #     neato_data.append(temp)
# # print(neato_data)
# print(len(token_reduced))

# dt = [('len', float)]

# A = np.array(neato_data).view(dt)
# G = nx.from_numpy_matrix(A)
# relabeled_nodes = {}
# for x, y in zip(range(len(neato_data[0])), TOKEN_NAMES):
#     relabeled_nodes[x] = y
#     # print(y[1:-1])
# G = nx.relabel_nodes(G, relabeled_nodes)

# G = to_agraph(G)

# G.node_attr.update(style='filled', fontsize=20)
# G.edge_attr.update(color='black', style='invis')
# print(G)
# #  ♭ ♮ ...for copy and paste
# G.draw('/home/daniel/Desktop/tryit.pdf', format='pdf', prog='neato')

In [ ]:
ESTTR11
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
import pygraphviz as pgv
labels = ['S', 'T', 'D']
G = nx.DiGraph()
for x, y in zip(ESTTR11, labels):
    for a, b in zip(x, labels):
        G.add_edge(y, b, penwidth=a*5)
edge_weights = []
edges = G.edges(data=True)
print('edges:', edges)
for x in edges:
    edge_weights.append(x[2]['penwidth'])
nodes = G.nodes()
print('nodes:', nodes)
for x in nodes:
    G.node[x]['shape'] = 'circle'
G = to_agraph(G)
# G=pgv.AGraph(ranksep=10)
G.draw('/home/daniel/Desktop/function.png', format='png', prog='dot')

In [ ]:
pp = FIT_DATA.predict_proba(at)
bigF = []
for x in range(len(pp)):
    counter = 0
    func = 0
    for y in pp[x]:
        if y >= .9:
            counter += 1
            func += func_guess[x]
    if counter > 0:
        bigF.append(func)
    else:
        bigF.append('-')

In [ ]:
a

In [22]:
alf_fit = joblib.load('/home/daniel/Desktop/hmm/alfabeto_continuo_major_3.pkl')
FIT_DATA = alf_fit['fit']
TOKEN_NAMES = alf_fit['tokens']
felici_numerals = transposed_pc_chords_noMMD(di.GetComposer.all_kaps[3][10]['alfabeto'], 'all')
felici_numb = [[TOKEN_NAMES.index(i)] for i in felici_numerals]
# f_nums = di.python_hmm(felici, di.modal_major)
f_guess = FIT_DATA.predict(felici_numb)
pp = FIT_DATA.predict_proba(felici_numb)
Labels = ['T', 'D', 'S']

In [ ]:
bigF = []
reduced_numerals = []
bigF2 = []
for x in range(len(pp)):
    counter = 0
    func = 0
    for y in pp[x]:
        if y >= .95:
            counter += 1
            func += f_guess[x]
            reduced_numerals.append(felici_numb[x])
    if counter > 0:
        bigF.append(Labels[func])
    else:
        bigF.append('-')
func_surface = [Labels[n] for n in f_guess]


In [ ]:
csv_felici = [felici_numerals, f_guess, [f[0] for f in pp], [f[1] for f in pp], [f[2] for f in pp]]

In [ ]:
csv_function(csv_felici, '/home/daniel/Desktop/hierarchy.csv')

In [ ]:
def hierarchy_maker(corpus_function):
    f_numbers = [[TOKEN_NAMES.index(i)] for i in corpus_function]
    reduced_numerals = copy.deepcopy(f_numbers)
    rereduced_numerals = []
    print([TOKEN_NAMES[x[0]] for x in reduced_numerals])
    
    while reduced_numerals != rereduced_numerals:
        if len(rereduced_numerals) > 0:
            reduced_numerals = rereduced_numerals
        f_guess = FIT_DATA.predict(reduced_numerals)
        pp = FIT_DATA.predict_proba(reduced_numerals)
        print(len(pp))
        rereduced_numerals = []
        for x in range(len(pp)):
            counter = 0
            func = 0
            for y in pp[x]:
                if y >= .99:
                    counter += 1
                    func += f_guess[x]
                    rereduced_numerals.append(reduced_numerals[x])
#         rereduced_numerals = []
        print([TOKEN_NAMES[x[0]] for x in reduced_numerals])
        print(len(reduced_numerals))
    print()
#         if counter > 0:
#             bigF.append(Labels[func])
#         else:
#             bigF.append('-')
#     func_surface = [Labels[n] for n in f_guess]

In [ ]:
hierarchy_maker(transposed_pc_chords_noMMD(di.GetComposer.all_kaps[3][10]['alfabeto'], 'all'))

In [ ]:
hierarchy_maker

In [3]:
alf_fit = joblib.load('/home/daniel/Desktop/hmm/alfabeto_continuo_major_3.pkl')

In [7]:
alf_fit['tokens']

['I',
 'i',
 'I6',
 'i6',
 'I64',
 'i64',
 'II',
 'ii',
 'ii6',
 'bIII',
 'III',
 'iii',
 'iii6',
 'IV',
 'IV6',
 'iv',
 '#iv',
 'V',
 'V6',
 'v',
 'bVI',
 'VI',
 'vi',
 'vi6',
 'bVII',
 'bVII6',
 'VII',
 'vii']

In [21]:
felici_numerals

['I',
 'V',
 'IV',
 'V',
 'I',
 'bVII',
 'vi',
 'V',
 'IV',
 'ii',
 'IV',
 'VI',
 'II',
 'bVII',
 'vi',
 'v',
 'IV',
 'V',
 'I',
 'bVII',
 'vi',
 'v',
 'IV',
 'V',
 'I']

In [12]:
len(pp)

47

In [23]:
all_data = []
for num, prob in zip(felici_numerals, pp):
    crd = [num]+list(prob)
    all_data.append(crd)

In [24]:
csv_function(all_data, '/home/daniel/Desktop/felici_function.csv')